<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Reinforcement_Learning_with_Self_Play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # Import the functional module
import torch.optim as optim
import random

# Define the SimplePolicy class
class SimplePolicy(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimplePolicy, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)  # Fully connected layer

    def forward(self, x):
        return F.softmax(self.fc(x), dim=-1)  # Apply softmax to get action probabilities

# Define the play_game function
def play_game(policy1, policy2):
    state = torch.randn(1, 4)  # Example game state (random)
    action_probs1 = policy1(state)  # Get action probabilities from policy1
    action1 = action_probs1.multinomial(num_samples=1)  # Sample action from policy1
    action_probs2 = policy2(state)  # Get action probabilities from policy2
    action2 = action_probs2.multinomial(num_samples=1)  # Sample action from policy2
    reward = torch.tensor([1.0]) if action1 == action2 else torch.tensor([-1.0])  # Reward based on actions
    return reward, action_probs1, action1

# Example self-play loop
policy = SimplePolicy(input_dim=4, output_dim=2)  # Instantiate the policy model
optimizer = optim.Adam(policy.parameters(), lr=0.01)  # Define the optimizer

for _ in range(1000):
    opponent_policy = policy  # Use the same policy as the opponent
    reward, action_probs, action = play_game(policy, opponent_policy)  # Play the game
    loss = -torch.log(action_probs.squeeze(0)[action]) * reward  # Compute the loss
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagate the loss
    optimizer.step()  # Update the model parameters

print("Self-play training completed.")